In [1]:
import wfdb
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import h5py
from mne.filter import filter_data
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN, KMeans
from mne.filter import filter_data, notch_filter
from sklearn.calibration import calibration_curve
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, roc_curve, auc, recall_score, precision_score, f1_score, confusion_matrix

import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

all_data_path = r"E:\a天理本人相关\智慧医疗\PAF\coding\lj\all_data"
small_data_path = r"E:\a天理本人相关\智慧医疗\PAF\coding\lj\data"

# file_path = "Net1d_newGroup\PAF_net1d_Group2\list_data.h5"
# file_path = "Net1d_Split\PAF_net1d_Split1_5\list_data.h5"
firstNum = 67
file_path = f"Net1d_Split1_denoise\PAF_net1d_SplitDenoise_FirstNum-{firstNum}/list_data{firstNum}.h5"
file_paths = ["Net1d_Split1_denoise\PAF_net1d_SplitDenoise_FirstNum-16\list_data16.h5", "Net1d_Split1_denoise\PAF_net1d_SplitDenoise_FirstNum-17\list_data17.h5", "Net1d_Split1_denoise\PAF_net1d_SplitDenoise_FirstNum-43\list_data43.h5", "Net1d_Split1_denoise\PAF_net1d_SplitDenoise_FirstNum-67\list_data67.h5", "Net1d_Split1_denoise\PAF_net1d_SplitDenoise_FirstNum-103\list_data103.h5"]

hf_keys = []
data = {}
for file_path in file_paths:
    with h5py.File(file_path, 'r') as hf:
        for key in hf.keys():
            hf_keys.append(key)
            data[key] = hf[key][:]
print(hf_keys)
print(len(hf_keys))


people_info = pd.read_csv(r"E:\a天理本人相关\智慧医疗\PAF\coding\data_distil\people_info.csv")
key_value = people_info.set_index('Key').to_dict()['Value']
print(key_value)

['data_12_1', 'data_12_2', 'data_12_3', 'data_12_4', 'data_12_5', 'data_12_6', 'data_12_7', 'data_12_8', 'data_16_1', 'data_16_2', 'data_16_3', 'data_16_4', 'data_22_1', 'data_22_10', 'data_22_11', 'data_22_12', 'data_22_13', 'data_22_14', 'data_22_15', 'data_22_16', 'data_22_2', 'data_22_3', 'data_22_4', 'data_22_5', 'data_22_6', 'data_22_7', 'data_22_8', 'data_22_9', 'data_26_1', 'data_26_10', 'data_26_11', 'data_26_12', 'data_26_13', 'data_26_2', 'data_26_3', 'data_26_4', 'data_26_5', 'data_26_6', 'data_26_7', 'data_26_8', 'data_26_9', 'data_36_1', 'data_36_2', 'data_36_3', 'data_36_4', 'data_36_5', 'data_36_6', 'data_36_7', 'data_36_8', 'data_40_1', 'data_40_2', 'data_40_3', 'data_40_4', 'data_40_5', 'data_41_1', 'data_41_10', 'data_41_11', 'data_41_12', 'data_41_13', 'data_41_14', 'data_41_2', 'data_41_3', 'data_41_4', 'data_41_5', 'data_41_6', 'data_41_7', 'data_41_8', 'data_41_9', 'data_45_1', 'data_45_10', 'data_45_2', 'data_45_3', 'data_45_4', 'data_45_5', 'data_45_6', 'data_4

# 长图绘制

In [2]:
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# 修改渲染器为"vscode"
pio.renderers.default = "vscode"

def afDurationCreate(aux, afDuration):
    seg = []
    for i in range(len(aux)):
        if aux[i] == '(AFIB' or aux[i] == 'AFL':
            if len(seg) == 0:
                seg.append(i)
            else:
                continue
        elif aux[i] == '(N':
            if len(seg) != 0:
                seg.append(i)
            else:
                seg.append(i)
                seg.append(i)
            # seg.append(i)
        if i == len(aux) - 1 and len(seg) == 1:
            seg.append(i)
        if len(seg) == 2:
            afDuration.append(seg)
            seg = []
    if len(afDuration) == 0:
        afDuration.append([0, 0])
def checkInAf(x, afDuration):
    # print(afDuration)
    try:
        for temp in afDuration:
            s, e = temp[0], temp[1]
            if x >= s and x < e:
                return True
        return False
    except:
        print(afDuration)
        print(x)

targets = ['data_31_14']
# len(hf_keys): 1435
testSet = [4, 11, 17, 18, 29, 30, 31, 37, 48, 75, 77, 83, 84, 86, 88, 93, 95, 97, 100, 102, 104]
for target in targets[:1]:
    if int(target.split('_')[1]) not in testSet:
        continue
    file_name = target
    record = wfdb.rdrecord(os.path.join(all_data_path, file_name))
    sig, fields = wfdb.rdsamp(os.path.join(all_data_path, file_name))
    label = key_value[int(target.split('_')[1])]
    # if label == 1:
    #     continue

    annotation = wfdb.rdann(os.path.join(all_data_path, file_name), 'atr')
    R_location = annotation.sample
    R_symbol = annotation.symbol
    aux = annotation.aux_note
    p_signal = record.p_signal[:, 0]
    p_signal = p_signal.flatten()
    # print("R_symbol: ", R_symbol)

    # 滤波操作
    sfreq = 200
    low_freq = 0.5
    high_freq = 50.0
    p_signal = filter_data(p_signal, sfreq, low_freq, high_freq, method='iir', verbose=False)

    # 获得概率值
    seg_data = data[target]
    # print(seg_data.shape)
    seg_pro = []
    for i in range(len(seg_data)):
        seg_pro.append(round(seg_data[i][-2], 2))
    
    # AF_location = []
    # for i in range(len(R_symbol))[10:-5]:
    #     if R_symbol[i] != 'N':
    #         AF_location.append(i+1)
    
    afDuration = []
    afDurationCreate(aux, afDuration)

    # 概率变化情况，因只统计了N，所以要将我们的start和end转换为N的start和end
    all_seg_pro = [0 for i in range(10)]
    index = 0
    for i in range(len(R_symbol))[10:-5]:
        if R_symbol[i] == 'N':
            all_seg_pro.append(seg_pro[index])
            index += 1
        else:
            all_seg_pro.append(1)
    # 标记是否属于AF片段
    auxList = []
    for i in range(len(R_location)):
        mid_idx = R_location[i]
        if checkInAf(i, afDuration):
            auxList.append('AF')
        else:
            auxList.append('NN')
    
    print("afDuration: ", afDuration)
    
    df = pd.DataFrame(dict(
        x = range(10, len(p_signal)-5),
        y = p_signal[10:-5],
    ))
    df_annote = pd.DataFrame(dict(
        loc = R_location[10:-5],
        pro = all_seg_pro[10:],
        labels = R_symbol[10:-5],
        aux_note = auxList[10:-5]
            ))
    # print(df.head())
    # print(df_annote.head())
    fig = px.line(df, x = 'x', y = 'y')
    # Create the bar chart using Plotly Express
    bar_fig = px.bar(df_annote, x='loc', y='pro')

    # Update bar trace properties
    bar_fig.update_traces(marker_color='orange')  # Customize bar color if needed

    # Combine the two figures
    fig.add_trace(bar_fig.data[0])
    # Add scatter points for annotations from df_annote on the line plot
    # Add scatter points for 'labels' with custom color
    labels_trace = px.scatter(df_annote, x='loc', y='pro', text='labels')
    labels_trace.update_traces(marker=dict(size=8, color='blue'),
                               textfont=dict(color='blue'),
                                mode='markers+text',
                                textposition='top center')
    fig.add_trace(labels_trace.data[0])

    # Add scatter points for 'aux_note' with custom color and adjusted position
    aux_trace = px.scatter(df_annote, x='loc', y='pro', text='aux_note')
    aux_trace.update_traces(marker=dict(size=8, color='red'),
                            textfont=dict(color='red'),
                            mode='markers+text',
                            textposition='bottom center')
    fig.add_trace(aux_trace.data[0])
    fig.update_layout(width=1200)

    # Show the plot
    fig.show()
    # 蓝色表示beat级的标注，红色表示AF的标注，橙色表示概率值，当前是非房颤患者

afDuration:  [[463, 472]]
